In [38]:
import pandas as pd

In [39]:
school_data_file = "Resources/schools_complete.csv"
student_data_file = "Resources/students_complete.csv"

In [40]:
school_data = pd.read_csv(school_data_file)
student_data = pd.read_csv(student_data_file)

In [41]:
merged = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])
merged.head()

,Student ID,student_name,gender,grade,school_name,reading_score,math_score,School ID,type,size,budget
0,0,Paul Bradley,M,9th,Huang High School,66,79,0,District,2917,1910635
1,1,Victor Smith,M,12th,Huang High School,94,61,0,District,2917,1910635
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60,0,District,2917,1910635
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58,0,District,2917,1910635
4,4,Bonnie Ray,F,9th,Huang High School,97,84,0,District,2917,1910635


In [42]:
schools = merged["school_name"].unique()
print(schools)

['Huang High School' 'Figueroa High School' 'Shelton High School'
 'Hernandez High School' 'Griffin High School' 'Wilson High School'
 'Cabrera High School' 'Bailey High School' 'Holden High School'
 'Pena High School' 'Wright High School' 'Rodriguez High School'
 'Johnson High School' 'Ford High School' 'Thomas High School']


In [43]:
num_schools = int(len(schools))

In [44]:
num_students = int(merged["student_name"].count())

In [45]:
budget = merged["budget"].unique()
total_budget = float(sum(budget))

In [46]:
math = merged["math_score"]
math_score = float(math.mean())

In [47]:
reading = merged["reading_score"]
reading_score = float(reading.mean())

In [48]:
passing_math = merged.loc[(math >= 70), :]
pass_math = passing_math["Student ID"].count()

In [49]:
passing_reading = merged.loc[(reading >= 70), :]
pass_read = passing_reading["Student ID"].count()

In [50]:
percent_pass_math = (pass_math / num_students) * 100

In [51]:
percent_pass_read = (pass_read / num_students) * 100

In [52]:
overall_passing = merged.loc[(math >= 70) & (reading >= 70)]
overall_passing["Student ID"].head()
overall_passing_num = overall_passing["Student ID"].count()
percent_passing = (overall_passing_num / num_students) * 100

In [53]:
district_summary = {
    "Total Schools": num_schools, 
    "Total Students": num_students, 
    "Total Budget": total_budget,
    "Average Math Score": math_score,
    "Average Reading Score": reading_score,
    "Percent Passing Math": percent_pass_math,
    "Percent Passing Reading": percent_pass_read,
    "Percent Passing Overall": percent_passing
}
district_summary_df = pd.DataFrame(district_summary, index=[0])
district_summary_df.head()

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,Percent Passing Math,Percent Passing Reading,Percent Passing Overall
0,15,39170,24649428.0,78.985371,81.87784,74.980853,85.805463,65.172326


In [54]:
#formatting
district_summary_df["Total Students"] = district_summary_df["Total Students"].map("{:,}".format)
district_summary_df["Total Budget"] = district_summary_df["Total Budget"].astype(float).map("${:,.2f}".format)
district_summary_df.head()

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,Percent Passing Math,Percent Passing Reading,Percent Passing Overall
0,15,"39,170","$24,649,428.00",78.985371,81.87784,74.980853,85.805463,65.172326


In [55]:
school_passing_math = merged.loc[(math >= 70), :]
school_pass_math = school_passing_math.groupby("school_name").student_name.count()
school_passing_reading = merged.loc[(reading >= 70), :]
school_pass_read = school_passing_reading.groupby("school_name").student_name.count()

In [56]:
school_num_students = merged.groupby("school_name").student_name.count()

In [57]:
school_overall_passing = merged.loc[(math >= 70) & (reading >= 70)]
school_overall_passing_num = school_overall_passing.groupby("school_name").student_name.count()
school_percent_passing = (school_overall_passing_num / school_num_students) * 100

In [58]:
school_avg_budget = [merged.groupby("school_name").budget.mean()]

In [59]:
school_avg_math_score = merged.groupby("school_name").math_score.mean()

In [60]:
school_avg_read_score = merged.groupby("school_name").reading_score.mean()

In [61]:
schools_df = pd.DataFrame(merged.groupby("school_name").mean())
schools_df.head()

,Student ID,reading_score,math_score,School ID,size,budget
school_name,,,,,,
Bailey High School,20358.5,81.033963,77.048432,7.0,4976.0,3124928.0
Cabrera High School,16941.5,83.975780,83.061895,6.0,1858.0,1081356.0
Figueroa High School,4391.0,81.158020,76.711767,1.0,2949.0,1884411.0
Ford High School,36165.0,80.746258,77.102592,13.0,2739.0,1763916.0
Griffin High School,12995.5,83.816757,83.351499,4.0,1468.0,917500.0


In [62]:
school_type = merged.groupby("school_name").type.unique()

In [63]:
per_student_budget = (schools_df["budget"])/school_num_students

In [64]:
del schools_df["Student ID"]
del schools_df["School ID"]

In [65]:
new_schools_df = schools_df.rename(columns=
{
    "school_name": "School Name",
    "reading_score": "Average Reading Score",
    "math_score": "Average Math Score",
    "budget": "Total School Budget",
    "size": "Total Students"
})

In [108]:
new_schools_df["School Type"] = school_type 
new_schools_df["Total Students"] = school_num_students
new_schools_df["Per Student Budget"] = per_student_budget
new_schools_df["% Passing Math"] = (school_pass_math/school_num_students)*100
new_schools_df["% Passing Reading"] = (school_pass_read/school_num_students)*100
new_schools_df["% Overall Passing"] = school_percent_passing

In [67]:
final_schools_df = new_schools_df[["School Type","Total Students","Total School Budget","Per Student Budget","Average Math Score","Average Reading Score","% Passing Math","% Passing Reading","% Overall Passing"]]
final_schools_df["Per Student Budget"] = final_schools_df["Per Student Budget"].map("${:,.2f}".format)
final_schools_df["Total School Budget"] = final_schools_df["Total School Budget"].astype(float).map("${:,.2f}".format)
final_schools_df["School Type"] = final_schools_df["School Type"].str[0]
final_schools_df

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
school_name,,,,,,,,,
Bailey High School,District,4976,"$3,124,928.00",$628.00,77.048432,81.033963,66.680064,81.933280,54.642283
Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.061895,83.975780,94.133477,97.039828,91.334769
Figueroa High School,District,2949,"$1,884,411.00",$639.00,76.711767,81.158020,65.988471,80.739234,53.204476
Ford High School,District,2739,"$1,763,916.00",$644.00,77.102592,80.746258,68.309602,79.299014,54.289887
Griffin High School,Charter,1468,"$917,500.00",$625.00,83.351499,83.816757,93.392371,97.138965,90.599455
Hernandez High School,District,4635,"$3,022,020.00",$652.00,77.289752,80.934412,66.752967,80.862999,53.527508
Holden High School,Charter,427,"$248,087.00",$581.00,83.803279,83.814988,92.505855,96.252927,89.227166
Huang High School,District,2917,"$1,910,635.00",$655.00,76.629414,81.182722,65.683922,81.316421,53.513884
Johnson High School,District,4761,"$3,094,650.00",$650.00,77.072464,80.966394,66.057551,81.222432,53.539172


In [126]:
top_performers_df = final_schools_df.sort_values("% Overall Passing", ascending=False)
top_performers_df.head()

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
school_name,,,,,,,,,
Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.061895,83.975780,94.133477,97.039828,91.334769
Thomas High School,Charter,1635,"$1,043,130.00",$638.00,83.418349,83.848930,93.272171,97.308869,90.948012
Griffin High School,Charter,1468,"$917,500.00",$625.00,83.351499,83.816757,93.392371,97.138965,90.599455
Wilson High School,Charter,2283,"$1,319,574.00",$578.00,83.274201,83.989488,93.867718,96.539641,90.582567
Pena High School,Charter,962,"$585,858.00",$609.00,83.839917,84.044699,94.594595,95.945946,90.540541


In [127]:
bottom_performers_df = final_schools_df.sort_values("% Overall Passing", ascending=True)
bottom_performers_df.head()

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
school_name,,,,,,,,,
Rodriguez High School,District,3999,"$2,547,363.00",$637.00,76.842711,80.744686,66.366592,80.220055,52.988247
Figueroa High School,District,2949,"$1,884,411.00",$639.00,76.711767,81.158020,65.988471,80.739234,53.204476
Huang High School,District,2917,"$1,910,635.00",$655.00,76.629414,81.182722,65.683922,81.316421,53.513884
Hernandez High School,District,4635,"$3,022,020.00",$652.00,77.289752,80.934412,66.752967,80.862999,53.527508
Johnson High School,District,4761,"$3,094,650.00",$650.00,77.072464,80.966394,66.057551,81.222432,53.539172


In [70]:
ninth_grade = merged.loc[(merged["grade"] == "9th"), :]
tenth_grade = merged.loc[(merged["grade"] == "10th"), :]
eleventh_grade = merged.loc[(merged["grade"] == "11th"), :]
twelfth_grade = merged.loc[(merged["grade"] == "12th"), :]
ninth_grade_math = ninth_grade.groupby("school_name").math_score.mean()
tenth_grade_math = tenth_grade.groupby("school_name").math_score.mean()
eleventh_grade_math = eleventh_grade.groupby("school_name").math_score.mean()
twelfth_grade_math = twelfth_grade.groupby("school_name").math_score.mean()

In [71]:
math_by_grade = pd.concat({"9th":ninth_grade_math,
        "10th":tenth_grade_math,
        "11th":eleventh_grade_math,
        "12th":twelfth_grade_math}, axis=1)
math_by_grade

,9th,10th,11th,12th
school_name,,,,
Bailey High School,77.083676,76.996772,77.515588,76.492218
Cabrera High School,83.094697,83.154506,82.765560,83.277487
Figueroa High School,76.403037,76.539974,76.884344,77.151369
Ford High School,77.361345,77.672316,76.918058,76.179963
Griffin High School,82.044010,84.229064,83.842105,83.356164
Hernandez High School,77.438495,77.337408,77.136029,77.186567
Holden High School,83.787402,83.429825,85.000000,82.855422
Huang High School,77.027251,75.908735,76.446602,77.225641
Johnson High School,77.187857,76.691117,77.491653,76.863248


In [72]:
ninth_grade_read = ninth_grade.groupby("school_name").reading_score.mean()
tenth_grade_read = tenth_grade.groupby("school_name").reading_score.mean()
eleventh_grade_read = eleventh_grade.groupby("school_name").reading_score.mean()
twelfth_grade_read = twelfth_grade.groupby("school_name").reading_score.mean()

In [73]:
read_by_grade = pd.concat({"9th":ninth_grade_read,
        "10th":tenth_grade_read,
        "11th":eleventh_grade_read,
        "12th":twelfth_grade_read}, axis=1)
read_by_grade

,9th,10th,11th,12th
school_name,,,,
Bailey High School,81.303155,80.907183,80.945643,80.912451
Cabrera High School,83.676136,84.253219,83.788382,84.287958
Figueroa High School,81.198598,81.408912,80.640339,81.384863
Ford High School,80.632653,81.262712,80.403642,80.662338
Griffin High School,83.369193,83.706897,84.288089,84.013699
Hernandez High School,80.866860,80.660147,81.396140,80.857143
Holden High School,83.677165,83.324561,83.815534,84.698795
Huang High School,81.290284,81.512386,81.417476,80.305983
Johnson High School,81.260714,80.773431,80.616027,81.227564


In [181]:
# Scores by budget
bins = [0, 585, 630, 645, 680]
group_names = ["<$585","$585-630","$630-645","$645-680"]
scores_by_spending = pd.cut(per_student_budget, bins, labels=group_names, include_lowest=True)
columns = ["Average Math Score","Average Reading Score","% Passing Math","% Passing Reading","% Overall Passing"]

In [182]:
spending_df = final_schools_df.groupby(scores_by_spending).mean()
pd.options.display.float_format = '{:,.2f}'.format
spending_df = spending_df[columns]
spending_df.index.name="Spending Ranges (Per Student)"
spending_df.head()

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Spending Ranges (Per Student),,,,,
<$585,83.46,83.93,93.46,96.61,90.37
$585-630,81.90,83.16,87.13,92.72,81.42
$630-645,78.52,81.62,73.48,84.39,62.86
$645-680,77.00,81.03,66.16,81.13,53.53


In [147]:
# Scores by School Size
school_bins = [0, 1000, 2000, 5000]
label_names = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]
scores_by_school_size = pd.cut(school_num_students, school_bins, labels=label_names, include_lowest=True)

In [183]:
school_size_df = final_schools_df.groupby(scores_by_school_size).mean()
school_size_df = school_size_df[columns]
school_size_df.index.name="Scores by School Size"
school_size_df.head()

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Scores by School Size,,,,,
Small (<1000),83.82,83.93,93.55,96.10,89.88
Medium (1000-2000),83.37,83.86,93.60,96.79,90.62
Large (2000-5000),77.75,81.34,69.96,82.77,58.29


In [187]:
# Scores by School Type
school_type_df = final_schools_df.groupby("School Type").mean()
school_type_df = school_type_df[columns]
school_type_df.head()

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
School Type,,,,,
C,83.47,83.90,93.62,96.59,90.43
D,76.96,80.97,66.55,80.80,53.67
